In [ ]:
import pickle
import pandas as pd
import os
import glob
from IPython.display import display
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
folder_path = "results/final_runs/toulouse_case_cap7_updated_20_fisher_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1.0_use_AADMM-False_receding_20250409_100318/results/"
# Original case: # folder_path = "results/final_runs/toulouse_case_cap7_updated_20_fisher_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1.0_receding_20250304_095807/results/"
pattern = re.compile(r'^results_table_\d+\.csv$')
file_list = [f for f in os.listdir(folder_path) if pattern.match(f)]
auction_times = [int(file.split('_')[2][:-4]) + 20 for file in file_list]
sorted_auction_times = np.sort(auction_times)
sorted_file_list = [f for _, f in sorted(zip(auction_times, file_list))]
summary_data = {}

starting_capacity_list, capacity_list, prices_list, goods_list = [], [], [], []

path_list = [os.path.join(folder_path, file) for file in sorted_file_list]
df_list = [pd.read_csv(path) for path in path_list]

ind = 0
rebases_per_agent = {}
delay_vals = []
for df_results, auction_time, file_name in zip(df_list, sorted_auction_times, sorted_file_list):

    auction_number = int(auction_time/20)
    # Construct the new folder path
    # new_folder_path = os.path.join(file, "results")
    # results_table_path = os.path.join(new_folder_path, "market_results_table.csv")
    # performance_table_path = os.path.join(new_folder_path, "market_performance_table.csv")
    # auction_time = int(file.split('_')[2][:-4])

    # results_table_path = os.path.join(folder_path, file)
    pickle_file = os.path.join(folder_path, f"fisher_data_after_{auction_time - 20}.pkl")
    with open(pickle_file, 'rb') as f:
        data = pickle.load(f)

    pickle_file = os.path.join(folder_path, f"fisher_data_{auction_time - 20}.pkl")
    with open(pickle_file, 'rb') as f:
        fisher_data = pickle.load(f)
    
    starting_capacity_list.append(fisher_data['capacity'])
    capacity_list.append(data["market_data"]['capacity'])
    prices_list.append(data["market_data"]['prices'])
    goods_list.append(data["market_data"]['goods_list'])

    try:
        num_agents = len(df_results)
        # df_performance = pd.read_csv(performance_table_path)
        
        # Extract the capacity value from the folder name
        # capacity = int(file.split('_cap')[1].split('_')[0])
        
        # Count each unique value under the column 'Status'
        status_counts = df_results['Status'].value_counts().to_dict()
        
        # Get the number of agents
        # num_agents = df_performance['Number_Agents'].iloc[0]
        
        # Prepare column names specific to this file
        # file_key = f"{file}_cap{auction_time}"
        
        
        # Add unknown status categories dynamically
        for status, count in status_counts.items():
            if status not in ["allocated", "delayed", "dropped", "rebased", "parked"]:
                summary_data[file_name][f"Unknown_{status}"] = count


        df_results["Delay"] = df_results.apply(
            lambda row: int(row["Allocated Time (Arrival, Departure)"].split(" ")[1][:-1].replace("'", "")) - 
                int(row["Requested Time (Arrival, Departure)"].split(" ")[1][:-1].replace("'", "")) if row["Status"] == "delayed" else 0,
            axis=1)

        dropped, rebased_1, rebased_2 = 0, 0, 0
        for aircraft, group in df_results.groupby('Aircraft'):
            # Reset index within the group so we can use integer positions
                group = group.reset_index(drop=True)
                for i, row in group.iterrows():
                    if row['Status'] == 'rebased':
                        if ind >= 2:
                            previous_df_2 = df_list[ind-2]
                            if aircraft in previous_df_2['Aircraft'].values: 
                                print(f"Rebased aircraft {aircraft} was dropped")
                                dropped += 1
                                rebases_per_agent[aircraft] = 2
                                continue
                        if ind >= 1:
                            previous_df_1 = df_list[ind-1]
                            if aircraft in previous_df_1['Aircraft'].values: 
                                print(f"Rebased aircraft {aircraft} was previously rebased")
                                rebased_2 += 1
                                rebases_per_agent[aircraft] = 2
                                continue
                        print(f"Rebased aircraft {aircraft} is rebasing for the first time")
                        rebases_per_agent[aircraft] = 1
                        rebased_1 += 1
                        continue
                    elif row['Status'] == 'delayed':
                        delay_vals.append(row['Delay'])

        # Store data in a dictionary format
        summary_data[auction_number] = {
            # 'Capacity': capacity,
            # 'Number_Agents': num_agents,
            'Allocated': status_counts.get('allocated', 0),
            'Delayed': status_counts.get('delayed', 0),
            'Dropped': dropped,
            'Rebased Once': rebased_1,
            'Rebased Twice': rebased_2,
            'Rebased': rebased_1 + rebased_2,
            "Parked": status_counts.get('parked', 0)
        }


                
        # summary_data[file_key]['Check'] = (
        #     summary_data[file_key]['Allocated'] +
        #     summary_data[file_key]['Delayed'] +
        #     summary_data[file_key]['Dropped'] +
        #     summary_data[file_key]['Rebased'] +
        #     summary_data[file_key]['Parked'] == num_agents
        # )
        
        summary_data[auction_number]["Percentage_Allocated"] = summary_data[auction_number]["Allocated"] / num_agents * 100
        summary_data[auction_number]["Percentage_Delayed"] = summary_data[auction_number]["Delayed"] / num_agents * 100
        summary_data[auction_number]["Percentage_Dropped"] = summary_data[auction_number]["Dropped"] / num_agents * 100
        summary_data[auction_number]["Percentage_Rebased"] = summary_data[auction_number]["Rebased"] / num_agents * 100
        summary_data[auction_number]["Percentage_Parked"] = summary_data[auction_number]["Parked"] / num_agents * 100
        summary_data[auction_number]["perfect_dropped_rebased"] = (summary_data[auction_number]["Dropped"] + summary_data[auction_number]["Rebased"]) / num_agents * 100
        
    except FileNotFoundError as e:
        print(f"File not found: {e}")
    except pd.errors.EmptyDataError as e:
        print(f"File is empty: {e}")
    display(df_results)
    ind += 1

print(f"Number of rebased agents: {len(rebases_per_agent)}")
print(f"Number of times rebased: {sum(rebases_per_agent.values())}")
print(f"Average number of rebases per agent: {sum(rebases_per_agent.values()) / len(rebases_per_agent)}")
print(f"Average delay: {np.mean(delay_vals)}")
print(f"Number delayed: {len(delay_vals)}")

# Convert dictionary to DataFrame
if summary_data:
    df_status_summary = pd.DataFrame.from_dict(summary_data, orient='index')
    df_status_summary.fillna(0, inplace=True)
    display(df_status_summary)
else:
    print("No valid data collected.")

In [ ]:
columns = ["Allocated", "Delayed", "Dropped", "Rebased Once", "Rebased Twice"]
color_map = {"Allocated": "purple", "Delayed": "blue", "Dropped": "green", "Rebased Once": "orange", "Rebased Twice": "brown"}
df_plot = df_status_summary[columns].sort_index()
fig, ax = plt.subplots(figsize=(6, 6))
df_plot.plot(kind='bar', stacked=True, ax=ax, zorder=3)
ax.grid(True, which="both", linestyle="--", linewidth=0.5, zorder=0)  # Grid for better visualization
# Labels and Title
ax.set_xlabel("Auction Number")
ax.set_ylabel("Number of Agents")
# ax.set_title("Allocations per Auction")
ax.legend(title="Status")

ax.tick_params(axis='x', rotation=0)

plt.savefig(f"{folder_path}/allocations_per_auction_with_rebase_split.png")
plt.show()

In [ ]:
## Calculate MCE with Final Allocation

# Find Leftover Resources for Each Auction
MCE_list = []
for cap, start_cap, price, goods in zip(capacity_list, starting_capacity_list, prices_list, goods_list):
    # remaining_capacity = data["market_data"]["capacity"][:-2].copy()
    # goods_list = data["market_data"]["goods_list"]

    # for agent, agent_data in data["agents_data"].items():
    #     def get_good_allocation(good, agent_data):
    #         agent_goods_list = agent_data["agent_goods_list"]
    #         final_allocation = agent_data["final_allocation"]
    #         if good in agent_goods_list:
    #             loc = agent_goods_list.index(good)
    #             allocation = final_allocation[loc]
    #             return allocation
    #         return 0
    #     full_allocation = [get_good_allocation(good, agent_data) for good in goods_list[:-2]]
    #     remaining_capacity -= full_allocation
    #     print(f"Leftover Resources: {remaining_capacity}")

    total_capacity = sum([start_cap[ind] for ind, good in enumerate(goods) if good[0][0] == "V" or good[0].split("_")[0] == good[1].split("_")[0]])

    # Sum Leftover Resources for Goods Priced Greater than Zero
    non_zero_prices = np.where(price[:-2] > 0.001)[0]
    for ind in non_zero_prices:
        print(f"Good {goods[ind]} has price {price[ind]}")
    print(non_zero_prices)
    extra_capacity = [cap[ind] for ind in np.where(price[:-2] > 0.001)[0]]
    MCE = sum(extra_capacity)/total_capacity*100
    MCE_list.append(MCE)
    print(f"Market Clearing Error: {MCE}")


plt.figure(figsize=(6, 6))
plt.grid(True)
plt.plot(list(range(1, len(MCE_list)+1)), MCE_list, marker='o', linestyle="--")
plt.xlabel("Auction Number")
plt.ylabel("Market Clearing Error %")
plt.title("Market Clearing Error After Fractional Allocation")

plt.savefig(f"{folder_path}/market_clearing_error_after_fisher.png")

In [ ]:
## Number of Iterations vs I
import math

folder_path = "results/final_runs/toulouse_case_cap7_updated_20_fisher_b-50.0_agentsall_dval40.0_outval1.0_pout1.0_freq30.0_pbound3000.0_beta-method-none_alpha-1.0_receding_20250304_095807/results/"
folder_path = "results/final_runs/auction_frequency_cap10/"

dfs = []
I_values = []
for folder in os.listdir(folder_path):
    if not folder.startswith("t"):
        continue
    sub_folder_path = os.path.join(folder_path, f"{folder}/results")
    period = int(folder.split("_")[4][5:])

    # Check if it's a directory
    if os.path.isdir(sub_folder_path):
        csv_path = os.path.join(sub_folder_path, "market_performance_table.csv")

        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path)
            df["I"] = int(400/period)
            I_values.append(int(400/period))
            dfs.append(df)
combined_df = pd.concat(dfs, ignore_index=True)         
data = [combined_df[combined_df['I'] == I]['Number_Interations'].values for I in I_values]
data = [[val for val in row if not math.isnan(val)] for row in data]
print(data)

plt.figure(figsize=(6, 6))

plt.grid(True, which="both", linestyle="--", linewidth=0.5)  # Grid for better visualization
meanprops = dict(markeredgecolor='red', markersize=10, linestyle='-', linewidth=2, color='red')
plt.boxplot(data, positions=I_values, widths=2, showfliers=False, patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='blue'),
            capprops=dict(color='blue'),
            whiskerprops=dict(color='blue'),
            flierprops=dict(color='blue', markeredgecolor='blue'),
            medianprops=meanprops)
# combined_df = pd.concat(dfs, ignore_index=True)
# combined_df = combined_df.sort_values(by="I", key=lambda x: x.astype(float))
# plt.plot(df["I"], combined_df["Number_Interations"], marker='o')

plt.xlabel("Number of Auctions (I)", fontsize=12)
plt.ylabel("Number of Iterations", fontsize=12)
# plt.title("Number of Iterations Needed ", fontsize=14)
# plt.grid(True)
# filtered_df = filtered_df.sort_values(by="Freq", key=lambda x: x.astype(float))

# plt.plot(filtered_df["Freq"], filtered_df["Number_Interations"], marker='o')
# plt.show()
plt.savefig(f"{folder_path}/iter_vs_I.png")